# Prediction metrics

In [1]:
def LHT(op, l, r):
    if op in Locks[l]:
        return service_time[op] + lock_cost[l][PLACE][r]
    else:
        return 0

    
def op_serialization(l, r):
    res = 0
    for op in Ops:
        if modes[MODE][op] == 'X':
            for r_1 in Reps:
                if r_1 != r:
                    res += LHT(op, l, r_1) * freq[WL][op][r_1]
#     print('os', res)
    return res


def rep_serialization(l, r):
    res = 0
    for op in Ops:
        res += LHT(op, l, r) * freq[WL][op][r]
#     print('rp', res)
    return res


def op_parallelism(l, r):
    res = 0
    s = 0
    for r_1 in Reps:
        if r != r_1:
            for op in Ops:
                if modes[MODE][op] == 'S':
                    s += LHT(op, l, r_1) * freq[WL][op][r_1]
        res = max(res, s)
#     print('op', res)
    return res


def nonparallelism(l):
    res = 0
    for r in Reps:
        np = abs(op_serialization(l, r) + rep_serialization(l, r) - op_parallelism(l,r))
        res = max(res, np)
#     print('np', res)
    return res


def coordination_parallelism():
    res = 0
    for l in Locks:
        np = nonparallelism(l)
        res = max(res, np)
    return res


def max_replica_exec_time(WL, l):
    res = 0
    for r in Reps:
        repexec = rep_serialization(l, r)
        res = max(res, repexec)
    return res


In [2]:
Locks = {'a-b':{'insert', 'read'}}
Reps = ['houston', 'paris', 'singapore']
Ops = ['insert', 'read']
modes = {1:{'insert':'X', 'read':'X'}, 2:{'insert':'X','read':'S'}, 3:{'insert':'S','read':'X'}}
service_time = {'insert':0.5, 'read':0.5}
# lock_cost = {'a-b': {'cent':{'houston':115, 'paris':5, 'singapore':155}, 
            #'clust':{'houston':137, 'paris':137, 'singapore':287}}}
             #, 'dist':{'houston':115, 'paris':5, 'singapore':155}} #-- measure this
lock_cost = {'a-b': {'houston':{'houston':12.427988394, 'paris':349.75518603, 'singapore':769.29997514}, 
                     'paris':{'houston':349.3677700815, 'paris':12.254517579, 'singapore':469.4461964361}, 
                     'singapore':{'houston':772.737378671, 'paris':471.905044859, 'singapore':10.9485321825}}}
freq = {'workloadeqeq':{'insert':{'houston':167, 'paris':167, 'singapore':166}, 'read':{'houston':167, 'paris':167, 'singapore':166}},
       'workloadeqhot':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':0, 'paris':500, 'singapore':0}},
        'workloadeqclust':{'insert':{'houston':250, 'paris':250, 'singapore':0}, 'read':{'houston':250, 'paris':250, 'singapore':0}},
        'workloadhoteq':{'insert':{'houston':333, 'paris':334, 'singapore':333}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadhothot':{'insert':{'houston':0, 'paris':1000, 'singapore':0}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadhotclust':{'insert':{'houston':500, 'paris':500, 'singapore':0}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadF':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':167, 'paris':167, 'singapore':166}},
        'workloadG':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':250, 'paris':0, 'singapore':250}}
       }

In [3]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                print(max_replica_exec_time(WL, l))

255573.59174648
156022.1372167852
258261.284476114
255573.59174648
156022.1372167852
258261.284476114
255573.59174648
156022.1372167852
258261.284476114
350255.18603
12754.517579
472405.04485899996
350255.18603
12754.517579
472405.04485899996
350255.18603
12754.517579
472405.04485899996
175127.593015
174933.88504075
386618.6893355
175127.593015
174933.88504075
386618.6893355
175127.593015
174933.88504075
386618.6893355
256343.39172162
156492.08341322132
257488.047097443
256343.39172162
156492.08341322132
257488.047097443
256343.39172162
156492.08341322132
257488.047097443
350255.18603
12754.517579
472405.04485899996
350255.18603
12754.517579
472405.04485899996
350255.18603
12754.517579
472405.04485899996
175127.593015
174933.88504075
386618.6893355
175127.593015
174933.88504075
386618.6893355
175127.593015
174933.88504075
386618.6893355
233620.20908201
78011.0686083926
315094.164920953
233620.20908201
78011.0686083926
315094.164920953
233620.20908201
78011.0686083926
315094.164920953
1

In [4]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
#             print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', coordination_parallelism())
            print(coordination_parallelism())

376876.772004096
277137.9812953922
419845.48214361
255573.59174648003
156022.1372167852
258261.284476114
255573.59174648003
156022.1372167852
258261.284476114
350255.18603
12754.517579
472405.04485899996
350255.18603
12754.517579
472405.04485899996
350255.18603
12754.517579
472405.04485899996
181591.58721199998
181311.14383025
622821.211765
175127.593015
174933.88504075
386618.6893355
175127.593015
174933.88504075
386618.6893355
377633.643990842
277258.05972174683
419083.6932971215
377633.643990842
277258.05972174683
419083.6932971215
369023.603720438
268738.0419789748
411458.97086357645
350255.18603
12754.517579
472405.04485899996
350255.18603
12754.517579
472405.04485899996
350255.18603
12754.517579
472405.04485899996
181591.58721199998
181311.14383025
622821.211765
181591.58721199998
181311.14383025
622821.211765
181591.58721199998
181311.14383025
622821.211765
363565.979017048
144946.2494371961
446125.263501305
242262.798759432
127931.7229868101
284541.06583380897
363565.979017048


In [5]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                for r in Reps:
#                     print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', op_serialization(l, r))
                    print(op_serialization(l, r))

372558.8238805
259891.539870076
121303.180257616
160282.1460881712
272877.9724240062
121115.844078607
161584.197667496
262062.197160704
416044.56945902
186279.41194025
129945.769935038
60651.590128808
80141.0730440856
136438.9862120031
60557.9220393035
80792.098833748
131031.098580352
208022.28472951
186279.41194025
129945.769935038
60651.590128808
80141.0730440856
136438.9862120031
60557.9220393035
80792.098833748
131031.098580352
208022.28472951
350255.18603
0.0
350255.18603
12754.517579
0.0
12754.517579
472405.04485899996
0.0
472405.04485899996
175127.593015
0.0
175127.593015
6377.2587895
0.0
6377.2587895
236202.52242949998
0.0
236202.52242949998
175127.593015
0.0
175127.593015
6377.2587895
0.0
6377.2587895
236202.52242949998
0.0
236202.52242949998
175127.593015
6463.994197
181591.58721199998
6377.2587895
174933.88504075
181311.14383025
236202.52242949998
386618.6893355
622821.211765
87563.7965075
3231.9970985
90795.79360599999
3188.62939475
87466.942520375
90655.571915125
118101.26

In [7]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                for r in Reps:
#                     print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', rep_serialization(l, r))
                    print(rep_serialization(l, r))
    

4317.948123596
116985.23213402
255573.59174648
116855.835207221
4260.008871386
156022.1372167852
258261.284476114
157783.28498290599
3800.9126845899996
4317.948123596
116985.23213402
255573.59174648
116855.835207221
4260.008871386
156022.1372167852
258261.284476114
157783.28498290599
3800.9126845899996
4317.948123596
116985.23213402
255573.59174648
116855.835207221
4260.008871386
156022.1372167852
258261.284476114
157783.28498290599
3800.9126845899996
0.0
350255.18603
0.0
0.0
12754.517579
0.0
0.0
472405.04485899996
0.0
0.0
350255.18603
0.0
0.0
12754.517579
0.0
0.0
472405.04485899996
0.0
0.0
350255.18603
0.0
0.0
12754.517579
0.0
0.0
472405.04485899996
0.0
6463.994197
175127.593015
0.0
174933.88504075
6377.2587895
0.0
386618.6893355
236202.52242949998
0.0
6463.994197
175127.593015
0.0
174933.88504075
6377.2587895
0.0
386618.6893355
236202.52242949998
0.0
6463.994197
175127.593015
0.0
174933.88504075
6377.2587895
0.0
386618.6893355
236202.52242949998
0.0
4305.020135202
116985.23213402
256

In [8]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                for r in Reps:
#                     print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', op_parallelism(l, r))
                    print(op_parallelism(l, r))
                    

0
0
0
0
0
0
0
0
0
186279.41194025
129945.769935038
60651.590128808
80141.0730440856
136438.9862120031
60557.9220393035
80792.098833748
131031.098580352
208022.28472951
186279.41194025
129945.769935038
60651.590128808
80141.0730440856
136438.9862120031
60557.9220393035
80792.098833748
131031.098580352
208022.28472951
0
0
0
0
0
0
0
0
0
175127.593015
0
175127.593015
6377.2587895
0
6377.2587895
236202.52242949998
0
236202.52242949998
175127.593015
0
175127.593015
6377.2587895
0
6377.2587895
236202.52242949998
0
236202.52242949998
0
0
0
0
0
0
0
0
0
87563.7965075
3231.9970985
90795.79360599999
3188.62939475
87466.942520375
90655.571915125
118101.26121474999
193309.34466775
311410.6058825
87563.7965075
3231.9970985
90795.79360599999
3188.62939475
87466.942520375
90655.571915125
118101.26121474999
193309.34466775
311410.6058825
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
373328.62385564
260648.411856822
121290.252269222
160752.09228460732
272998.0508503608
120765.9763085255
161595.64619967848
261300.4